The original genus statics file was generated in notebook 08_O2_transcript_biogeochemistry_GEN_stats_master Now I am taking that file and the GEN_statics file and adding a couple of additional values to them.

In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob
from Bio import SeqIO


os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/')
images='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/'
outdir=('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/')
matplotlib.__version__

'3.3.3'

rerun on 2021-12-07 after correcting for salinity of media samples

In [2]:
Genera=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/GEN_statistics.csv', index_col=0)
Genera[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']]=Genera['full_GTDBtk_classification'].str.split(';', expand=True)
cols=['GTDB_genus', 'full_GTDBtk_classification', 'domain', 'phyla', 'class', 'order', 'family', 'genus', 'species', 
      '#_of_cells', 'genus_size_rank', 'Average_gc_content', 'Average_hypothetical_cds_fraction', 
      'median_estimated_diameter', 'mean_estimated_genome_length']

Genera=Genera[cols]

Genera[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']]=Genera['full_GTDBtk_classification'].str.split(';', expand=True)

cols=['Average_doubling_time', 'median_doubling_time', 'GTDB_genus']
growth=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genus_statistics.csv', index_col=0,
                  usecols=cols)
growth=growth.drop_duplicates()
Genera=Genera.merge(growth, on='GTDB_genus', how='left')
unc=Genera.iloc[0].copy()
Genera=Genera[Genera['GTDB_genus']!= 'Unclassified']
Genera['domain']=Genera['domain'].str[3:]
Genera['phyla']=Genera['phyla'].str[3:]
Genera['class']=Genera['class'].str[3:]
Genera['order']=Genera['order'].str[3:]
Genera['family']=Genera['family'].str[3:]
Genera['genus']=Genera['genus'].str[3:]
Genera['species']=Genera['species'].str[3:]

Genera=Genera.append(unc)
Genera.to_csv(outdir+'Supplemental_table_genus_statistics.csv')

### Future Additions
1. Add the detection of rhodopsin
2. Add the detection of bacteriochlorophyl


In [3]:
! grep "rhodopsin" /mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv > /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/tmp/rhodopsin.tsv
cols=pd.read_csv('/mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv', sep='\t',  nrows=0).columns.tolist()
rhodopsin=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/tmp/rhodopsin.tsv', sep='\t', names=cols)

cols=['name', 'GTDB_classification']
classification=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv', 
                           usecols=cols)

rhodopsin=rhodopsin.merge(classification, left_on='sag', right_on='name', how='left')

# create a list of all genera where rhodopsin was detected
rhodopsin_genera=rhodopsin[['GTDB_classification']].copy()
rhodopsin_genera=rhodopsin_genera.drop_duplicates()
rhodopsin_genera['Rhodopsin detected']='Yes'
rhodopsin_genera=rhodopsin_genera.rename(columns={'GTDB_classification':'GTDB_genus'})
rhodopsin_genera

,GTDB_genus,Rhodopsin detected
0,Hel1-33-131,Yes
1,IMCC9063,Yes
2,Thioglobus,Yes
3,Unclassified,Yes
5,SGZJ01,Yes
...,...,...
1609,UBA1268,Yes
1621,Opitutaceae,Yes
1623,Mariniblastus,Yes
1625,Arctic95D-9,Yes


In [4]:
! grep "chlorophyll" /mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv > /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/tmp/bacteriochlorophyll.tsv
cols=pd.read_csv('/mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv', sep='\t',  nrows=0).columns.tolist()
bacteriochlorophyll=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/tmp/bacteriochlorophyll.tsv', sep='\t', names=cols)

cols=['name', 'GTDB_classification']
classification=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv', 
                           usecols=cols)

bacteriochlorophyll=bacteriochlorophyll.merge(classification, left_on='sag', right_on='name', how='left')

# create a list of all genera where chlorophyll was detected
bacteriochlorophyll_genera=bacteriochlorophyll[['GTDB_classification']].copy()
bacteriochlorophyll_genera=bacteriochlorophyll_genera.drop_duplicates()
bacteriochlorophyll_genera['Chlorophyll detected']='Yes'
bacteriochlorophyll_genera=bacteriochlorophyll_genera.rename(columns={'GTDB_classification':'GTDB_genus'})
bacteriochlorophyll_genera

bacteriochlorophyll

,contig_id,sag,ncbi_id,crest_id,start,stop,strand,ec_number,prokka_function,prokka_gene,name,GTDB_classification
0,AH-135-A22_NODE_65,AH-135-A22,31989,31989,1009,2292,-,1.3.7.7,Light-independent_protochlorophyllide_reductas...,bchN,AH-135-A22,Planktomarina
1,AH-135-B10_NODE_92,AH-135-B10,200667,200667,95,1549,+,1.97.1.12,Photosystem_I_P700_chlorophyll_a_apoprotein_A2,psaB,AH-135-B10,UBA4465
2,AH-135-B13_NODE_44,AH-135-B13,135620,135620,8854,9696,-,1.97.1.12,Photosystem_I_P700_chlorophyll_a_apoprotein_A2,psaB,AH-135-B13,ASP10-02a
3,AH-135-B16_NODE_3,AH-135-B16,31989,31989,20464,21366,-,1.3.7.7,Light-independent_protochlorophyllide_reductas...,bchL,AH-135-B16,Octadecabacter
4,AH-135-B16_NODE_3,AH-135-B16,31989,31989,24916,26436,-,1.3.7.7,Light-independent_protochlorophyllide_reductas...,bchB,AH-135-B16,Octadecabacter
...,...,...,...,...,...,...,...,...,...,...,...,...
687,AH-709-M14_NODE_13,AH-709-M14,49546,49546,32448,33287,-,1.97.1.12,Photosystem_I_P700_chlorophyll_a_apoprotein_A2,psaB,AH-709-M14,Hel1-33-131
688,AH-709-M22_NODE_3,AH-709-M22,31989,31989,12365,13261,-,1.3.7.7,Light-independent_protochlorophyllide_reductas...,bchL,AH-709-M22,Planktomarina
689,AH-709-M22_NODE_3,AH-709-M22,31989,31989,16830,18374,-,1.3.7.7,Light-independent_protochlorophyllide_reductas...,bchB,AH-709-M22,Planktomarina
690,AH-709-M22_NODE_3,AH-709-M22,31989,31989,18371,19654,-,1.3.7.7,Light-independent_protochlorophyllide_reductas...,bchN,AH-709-M22,Planktomarina


In [5]:
# read genus statistics csv file and merge with the rhodopsin detection df. change NaN to No
Genera=pd.read_csv(outdir+'Supplemental_table_genus_statistics.csv', index_col=0)
Genera=Genera.merge(rhodopsin_genera, on='GTDB_genus', how='left')
Genera['Rhodopsin detected'].fillna('No', inplace=True)

# merge with the chlorophyll detection df. change NaN to No and write a new csv file
Genera=Genera.merge(bacteriochlorophyll_genera, on='GTDB_genus', how='left')
Genera['Chlorophyll detected'].fillna('No', inplace=True)
Genera.to_csv(outdir+'Supplemental_table_genus_statistics_light_metabolism.csv')
Genera

,GTDB_genus,full_GTDBtk_classification,domain,phyla,class,order,family,genus,species,#_of_cells,genus_size_rank,Average_gc_content,Average_hypothetical_cds_fraction,median_estimated_diameter,mean_estimated_genome_length,Average_doubling_time,median_doubling_time,Rhodopsin detected,Chlorophyll detected
0,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,NaN,628,1.0,29.846285,0.220466,0.300,1.464149e+06,8.115193,8.112808,Yes,No
1,SW10,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,SW10,NaN,326,2.0,50.074055,0.512254,0.895,4.920788e+06,4.386646,4.062906,Yes,Yes
2,SCGC-AAA076-P13,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,SCGC-AAA076-P13,NaN,178,3.0,34.074901,0.248851,0.315,1.930576e+06,8.312164,8.593175,Yes,No
3,D2472,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,D2472,NaN,173,4.0,37.613462,0.239195,0.360,2.020084e+06,9.696105,9.841434,Yes,No
4,Amylibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Amylibacter,NaN,159,5.0,37.020946,0.214869,0.540,2.787861e+06,5.397132,5.240596,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,UBA974,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,UA16,UBA974,NaN,1,299.0,40.153044,0.427273,0.520,2.418804e+06,5.094093,5.094093,Yes,No
299,UBP15,d__Bacteria;p__UBP15;c__;o__;f__;g__;s__,Bacteria,UBP15,NaN,NaN,NaN,NaN,NaN,1,300.0,55.561731,0.791301,1.930,1.499180e+07,NaN,NaN,No,No
300,Vibrio,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Vibrionaceae,Vibrio,Vibrio sp003856525,1,301.0,41.909604,0.206679,1.180,4.803806e+06,NaN,NaN,No,No
301,ZODW24,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,ZODW24,NaN,1,302.0,57.607054,NaN,0.710,3.014321e+06,NaN,NaN,No,No


### Adding spherical cell volume


In [6]:

import math
Genera['spherical cell volume (µm3)']=(4/3)*math.pi*(Genera['median_estimated_diameter']/2)**3
Genera.to_csv(outdir+'Supplemental_table_genus_statistics_light_metabolism_volume.csv')
Genera


,GTDB_genus,full_GTDBtk_classification,domain,phyla,class,order,family,genus,species,#_of_cells,genus_size_rank,Average_gc_content,Average_hypothetical_cds_fraction,median_estimated_diameter,mean_estimated_genome_length,Average_doubling_time,median_doubling_time,Rhodopsin detected,Chlorophyll detected,spherical cell volume (µm3)
0,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,NaN,628,1.0,29.846285,0.220466,0.300,1.464149e+06,8.115193,8.112808,Yes,No,0.014137
1,SW10,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,SW10,NaN,326,2.0,50.074055,0.512254,0.895,4.920788e+06,4.386646,4.062906,Yes,Yes,0.375377
2,SCGC-AAA076-P13,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,SCGC-AAA076-P13,NaN,178,3.0,34.074901,0.248851,0.315,1.930576e+06,8.312164,8.593175,Yes,No,0.016366
3,D2472,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,D2472,NaN,173,4.0,37.613462,0.239195,0.360,2.020084e+06,9.696105,9.841434,Yes,No,0.024429
4,Amylibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Amylibacter,NaN,159,5.0,37.020946,0.214869,0.540,2.787861e+06,5.397132,5.240596,Yes,No,0.082448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,UBA974,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,UA16,UBA974,NaN,1,299.0,40.153044,0.427273,0.520,2.418804e+06,5.094093,5.094093,Yes,No,0.073622
299,UBP15,d__Bacteria;p__UBP15;c__;o__;f__;g__;s__,Bacteria,UBP15,NaN,NaN,NaN,NaN,NaN,1,300.0,55.561731,0.791301,1.930,1.499180e+07,NaN,NaN,No,No,3.764181
300,Vibrio,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Vibrionaceae,Vibrio,Vibrio sp003856525,1,301.0,41.909604,0.206679,1.180,4.803806e+06,NaN,NaN,No,No,0.860290
301,ZODW24,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,ZODW24,NaN,1,302.0,57.607054,NaN,0.710,3.014321e+06,NaN,NaN,No,No,0.187402


I need to add avg 16S copies per cell as well as avg mRNA transcripts per cell to this file. 

I should probably seperate it out by day so 3 or 4 columns for each.

In [7]:
rRNA=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/rRNA_recruitment/16S_rRNA_copies_per_cell_per_Genus.csv')
rRNA=rRNA.rename(columns={'Genus':'GTDB_genus'})
rRNA

#mRNA-pd.read_csv('')

,GTDB_genus,16S_transcripts_per_cell_190709,16S_transcripts_per_cell_190402,16S_transcripts_per_cell_181030,16S_transcripts_per_cell_171102
0,AAA164-E04,140.764254,39.275440,981.667559,3515.741468
1,AAA536-G10,876.260279,1177.827026,737.519266,1822.436862
2,AG-337-I02,110.930849,189.338832,523.029913,1057.217128
3,AG-422-B15,98.039999,158.246223,89.546350,193.442284
4,ASP10-02a,867.171322,1130.112708,293.604448,1019.602490
...,...,...,...,...,...
107,UBA9320,87.200558,169.433003,108.222733,242.523706
108,UBA952,215.116643,256.637147,319.125965,433.627056
109,UBA9926,1221.920368,1944.103387,1104.796242,2430.373003
110,Unclassified,137.953673,105.084254,164.728499,565.254946


In [8]:
df=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/RNA_summaries/Prokka_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master.csv',
              index_col=[1,2])

df = df.loc[:, :'Weighted_avg_O2_consumed_per_cell']
df=df.iloc[:, :-1]
df=df.drop('Unnamed: 0' , axis='columns')
df
mRNA=pd.DataFrame()
mRNA['total_transcripts']=df.sum(axis=1)
mRNA=mRNA.reset_index()
mRNA['date']=mRNA['date'].astype(str)
mRNA.rename(columns={'genus':'GTDB_genus'}, inplace=True)
mRNA

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13622) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,GTDB_genus,date,total_transcripts
0,AAA164-E04,20171102,807.560509
1,AAA164-E04,20181030,123.023330
2,AAA164-E04,20190402,191.608116
3,AAA164-E04,20190709,200.913807
4,AAA536-G10,20171102,268.407780
...,...,...,...
659,Winogradskyella,20190709,24.121830
660,Yoonia,20171102,222.500742
661,Yoonia,20181030,84.185621
662,Yoonia,20190402,45.146774


In [9]:

pivot=mRNA.pivot_table(index='GTDB_genus', columns='date', values='total_transcripts')

pivot.rename(columns={'20171102':'20171102_avg_mRNA_transcripts_per_cell', '20181030':'20181030_avg_mRNA_transcripts_per_cell',
                      '20190402':'20190402_avg_mRNA_transcripts_per_cell', '20190709':'20190709_avg_mRNA_transcripts_per_cell'}, inplace=True)

pivot=pivot.merge(rRNA, on='GTDB_genus', how='outer')
pivot

,GTDB_genus,20171102_avg_mRNA_transcripts_per_cell,20181030_avg_mRNA_transcripts_per_cell,20190402_avg_mRNA_transcripts_per_cell,20190709_avg_mRNA_transcripts_per_cell,16S_transcripts_per_cell_190709,16S_transcripts_per_cell_190402,16S_transcripts_per_cell_181030,16S_transcripts_per_cell_171102
0,AAA164-E04,807.560509,123.023330,191.608116,200.913807,140.764254,39.275440,981.667559,3515.741468
1,AAA536-G10,268.407780,76.768780,130.086391,100.107579,876.260279,1177.827026,737.519266,1822.436862
2,AG-337-I02,1051.938131,73.557068,644.619482,699.528872,110.930849,189.338832,523.029913,1057.217128
3,AG-414-E02,356.902115,32.631727,41.584671,90.759588,NaN,NaN,NaN,NaN
4,AG-422-B15,239.596913,48.432920,32.480740,59.430572,98.039999,158.246223,89.546350,193.442284
...,...,...,...,...,...,...,...,...,...
178,S36-B12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,UBA11663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,UBA1515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,UBA3478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
Genera=Genera.merge(pivot, on='GTDB_genus', how='left')
Genera.to_csv(outdir+'Supplemental_table_genus_statistics_light_metabolism_volume_RNA.csv')
Genera

,GTDB_genus,full_GTDBtk_classification,domain,phyla,class,order,family,genus,species,#_of_cells,...,Chlorophyll detected,spherical cell volume (µm3),20171102_avg_mRNA_transcripts_per_cell,20181030_avg_mRNA_transcripts_per_cell,20190402_avg_mRNA_transcripts_per_cell,20190709_avg_mRNA_transcripts_per_cell,16S_transcripts_per_cell_190709,16S_transcripts_per_cell_190402,16S_transcripts_per_cell_181030,16S_transcripts_per_cell_171102
0,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,NaN,628,...,No,0.014137,661.725499,178.769940,200.334694,173.926632,320.126750,384.110523,313.271699,685.289686
1,SW10,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,SW10,NaN,326,...,Yes,0.375377,197.633621,68.510698,77.105177,48.602046,263.493554,48.266132,284.957910,828.838750
2,SCGC-AAA076-P13,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,SCGC-AAA076-P13,NaN,178,...,No,0.016366,306.045339,110.742132,162.810404,68.953993,337.086373,460.306328,244.451274,928.243856
3,D2472,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,D2472,NaN,173,...,No,0.024429,107.951627,36.615937,35.678578,44.640790,235.313944,417.082716,177.034930,495.045784
4,Amylibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Amylibacter,NaN,159,...,No,0.082448,374.071237,164.828336,154.779680,193.877090,983.333316,1482.368552,794.539005,1923.525106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,UBA974,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,UA16,UBA974,NaN,1,...,No,0.073622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,UBP15,d__Bacteria;p__UBP15;c__;o__;f__;g__;s__,Bacteria,UBP15,NaN,NaN,NaN,NaN,NaN,1,...,No,3.764181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,Vibrio,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Vibrionaceae,Vibrio,Vibrio sp003856525,1,...,No,0.860290,794.246348,241.021412,221.876387,254.693991,NaN,NaN,NaN,NaN
303,ZODW24,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,ZODW24,NaN,1,...,No,0.187402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
! grep "dsr" /mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv > /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/tmp/dsrA.tsv
# cols=pd.read_csv('/mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv', sep='\t',  nrows=0).columns.tolist()
# bacteriochlorophyll=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/tmp/bacteriochlorophyll.tsv', sep='\t', names=cols)

# cols=['name', 'GTDB_classification']
# classification=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv', 
#                            usecols=cols)

# bacteriochlorophyll=bacteriochlorophyll.merge(classification, left_on='sag', right_on='name', how='left')

# # create a list of all genera where chlorophyll was detected
# bacteriochlorophyll_genera=bacteriochlorophyll[['GTDB_classification']].copy()
# bacteriochlorophyll_genera=bacteriochlorophyll_genera.drop_duplicates()
# bacteriochlorophyll_genera['Chlorophyll detected']='Yes'
# bacteriochlorophyll_genera=bacteriochlorophyll_genera.rename(columns={'GTDB_classification':'GTDB_genus'})
# bacteriochlorophyll_genera

# bacteriochlorophyll

In [12]:
! head /mnt/scgc/simon/microg2p/Data/SAG_genes/faa/GORG_Maine2020_noGEN.fixed.tsv

contig_id	sag	ncbi_id	crest_id	start	stop	strand	ec_number	prokka_function	prokka_gene
AH-135-A01_NODE_1	AH-135-A01	54526	54526	636	854	+	.	hypothetical_protein	.
AH-135-A01_NODE_1	AH-135-A01	54526	54526	1149	2060	+	2.3.1.8	Phosphate_acetyltransferase	pta
AH-135-A01_NODE_1	AH-135-A01	54526	54526	2123	3409	+	1.1.1.308	Sulfopropanediol_3-dehydrogenase	hpsN
AH-135-A01_NODE_1	AH-135-A01	54526	54526	3411	4016	-	.	Arginine_exporter_protein_ArgO	argO
AH-135-A01_NODE_1	AH-135-A01	54526	54526	4026	5531	-	.	Glycine_betaine/choline_transport_system_permease_protein_OusW	ousW
AH-135-A01_NODE_1	AH-135-A01	54526	54526	5524	6573	-	.	Glycine_betaine/proline_betaine_transport_system_ATP-binding_protein_ProV	proV
AH-135-A01_NODE_1	AH-135-A01	54526	54526	6656	7618	-	.	Glycine_betaine-binding_periplasmic_protein_OusX	ousX
AH-135-A01_NODE_1	AH-135-A01	54526	54526	7716	9197	-	1.2.1.5	NADP/NAD-dependent_aldehyde_dehydrogenase_PuuC	puuC
AH-135-A01_NODE_1	AH-135-A01	54526	54526	9240	10238	-	.	hypothetical_prot